In [42]:
import boto3
import sagemaker
import sagemaker.session

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
default_bucket = sagemaker_session.default_bucket()

In [43]:
from sagemaker.workflow.parameters import (
    ParameterString,
)

task_token = ParameterString(
    name="TaskToken",
    default_value='NIL'
)

In [44]:
from sagemaker.lambda_helper import Lambda

# Lambda helper class can be used to create the Lambda function
func = Lambda(
    function_arn='arn:aws:lambda:ap-southeast-1:247544061391:function:test-callback-from-sagemaker-pipleline',
    execution_role_arn='arn:aws:iam::247544061391:role/service-role/test-execute-sagemaker-transform-pipleline-role-wl7ef0fl',
    timeout=120,
    memory_size=128,
)

In [45]:
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)

step_lambda_callbak = LambdaStep(
    name="LambdaStepCallbackToStepFunctions",
    lambda_func=func,
    inputs={
        "TaskToken": task_token
    },
    outputs=[],
)

In [46]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = f"TestPipelineCallbackStepFunctions"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        task_token,
    ],
    steps=[step_lambda_callbak],
)

In [47]:
import json
json.loads(pipeline.definition())

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'TaskToken',
   'Type': 'String',
   'DefaultValue': 'NIL'}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'LambdaStepCallbackToStepFunctions',
   'Type': 'Lambda',
   'Arguments': {'TaskToken': {'Get': 'Parameters.TaskToken'}},
   'FunctionArn': 'arn:aws:lambda:ap-southeast-1:247544061391:function:test-callback-from-sagemaker-pipleline',
   'OutputParameters': []}]}

In [48]:
pipeline.upsert(role_arn=role)

In [49]:
execution = pipeline.start()
execution.wait()

WaiterError: Waiter PipelineExecutionComplete failed: Waiter encountered a terminal failure state: For expression "PipelineExecutionStatus" we matched expected path: "Failed"